In [116]:
from pymongo import MongoClient
import pickle
import pandas as pd
import numpy as np
import cnfg
import requests
import json
from urlparse import urlparse

In [2]:
client = MongoClient()
db = client.music
s = db.songs
# counter
c = s.find()

In [111]:
config = cnfg.load(".genius_config")
client_key = config["client_key"]
client_secret = config["client_secret"]
access_token = 'uK247ank6jF7DY_E7TEMIdbuhT2IBVsVeFXh6AluxswFUfrlFNfXNu2pUFPrGv2c'
auth = 'Bearer '+access_token
req_start = 'https://api.genius.com'

In [3]:
def pickleLoad(filename):
    with open(filename, 'rb') as f:
        data = pickle.load(f) 
    return data

def pickleDump(data, filename):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)

In [113]:
def api_call(validurl):
    call = validurl
    r = requests.get(call, headers={'Authorization':auth})
    if r.status_code==200:
        return json.loads(r.content)
    else:
        print r.content
        return r.content

### Get artist to influencer dataframe

In [6]:
df = pickleLoad('./matrices/artist_to_influencer_1305x421.pkl')

In [34]:
sorted_related = pd.DataFrame(columns=df.columns)
sorted_neighbors = pd.DataFrame(columns=df.columns)

In [39]:
for artist in df:
    related = df[artist].sort_values(ascending=False, inplace=False)
    del related[artist]
    related = pd.DataFrame(related).reset_index()
    sorted_neighbors[artist] = related['index']
    sorted_related[artist] = related[artist]

In [42]:
best_neighbors = sorted_related.ix[0].sort_values(ascending=False, inplace=False)

In [54]:
num_neighbors = []
for a in sorted_related:
    num_neighbors.append((a, len([x for x in sorted_related[a] if x!=0])))
num_neighbors = sorted(num_neighbors, key=lambda x: -1*x[1])

In [71]:
nodes = [x[0] for x in num_neighbors[0:50]]
neighbors = [sorted_neighbors[x][0] for x in nodes]
total = list(set(nodes+neighbors))

In [77]:
len(total)

93

In [100]:
drake = 130

In [108]:
names = {"nodes":[]}

In [153]:
def format_artists(nodes, namedict):
    for artistid in nodes:
        if s.find({'artist_id':artistid}, {'artist':1}).count() >= 1:
            name = s.find({'artist_id':artistid}, {'artist':1}).next()
            namedict['nodes'].append({'name':name['artist'].encode('ascii', 'ignore')})
        else:
            url = req_start+'/artists/'+str(artistid)
            name = api_call(url)
            namedict['nodes'].append({'name':name['response']['artist']['name'].encode('ascii', 'ignore')})

In [154]:
format_artists(total, names)

In [155]:
names

In [151]:
id_to_name = {}

In [157]:
for artistid in total:
    if s.find({'artist_id':artistid}, {'artist':1}).count() >= 1:
        name = s.find({'artist_id':artistid}, {'artist':1}).next()
        name = name['artist'].encode('ascii', 'ignore')
        id_to_name[artistid] = name
    else:
        url = req_start+'/artists/'+str(artistid)
        r = api_call(url)
        name = r['response']['artist']['name'].encode('ascii', 'ignore')
        id_to_name[artistid] = name

In [160]:
info = names

In [171]:
links = []

In [179]:
indices = {}
for i, x in enumerate(names['nodes']):
    indicies[x['name']] = i

In [172]:
for i, x in enumerate(neighbors):
    links.append({'source':indices[id_to_name[x]],'target':indices[id_to_name[nodes[i]]],'value':1})

In [174]:
info['links'] = links

In [191]:
sorted(links)

[{'source': 0, 'target': 18, 'value': 1},
 {'source': 1, 'target': 80, 'value': 1},
 {'source': 3, 'target': 21, 'value': 1},
 {'source': 5, 'target': 20, 'value': 1},
 {'source': 7, 'target': 2, 'value': 1},
 {'source': 8, 'target': 62, 'value': 1},
 {'source': 12, 'target': 26, 'value': 1},
 {'source': 16, 'target': 30, 'value': 1},
 {'source': 17, 'target': 15, 'value': 1},
 {'source': 24, 'target': 22, 'value': 1},
 {'source': 29, 'target': 10, 'value': 1},
 {'source': 33, 'target': 65, 'value': 1},
 {'source': 34, 'target': 23, 'value': 1},
 {'source': 34, 'target': 24, 'value': 1},
 {'source': 37, 'target': 9, 'value': 1},
 {'source': 39, 'target': 72, 'value': 1},
 {'source': 39, 'target': 77, 'value': 1},
 {'source': 41, 'target': 31, 'value': 1},
 {'source': 42, 'target': 11, 'value': 1},
 {'source': 44, 'target': 6, 'value': 1},
 {'source': 45, 'target': 84, 'value': 1},
 {'source': 46, 'target': 66, 'value': 1},
 {'source': 46, 'target': 79, 'value': 1},
 {'source': 47, 'tar